In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from datetime import datetime
import re
from dateutil import parser

In [ ]:
convention_name_2018_2019 = Path("/Users/matt/Desktop/conventions_agh/convention_names/2018_2019_CONVENTION_NAME_2018_19.csv")
convention_name_2018_2019_df = pd.read_csv(convention_name_2018_2019, index_col = "Booked On", infer_datetime_format=True)
convention_name_2018_2019_df.head()

In [ ]:
convention_name_2020_2021 = Path("/Users/matt/Desktop/conventions_agh/convention_names/2020_2021_CONVENTION_NAME_2020_2021.csv")
convention_name_2020_2021_df = pd.read_csv(convention_name_2020_2021, index_col = "Booked On", infer_datetime_format=True)
convention_name_2020_2021_df.head()

In [ ]:
convention_name_2022_2023 = Path("/Users/matt/Desktop/conventions_agh/convention_names/2022_2023_CONVENTIONS_NAME_2022_2023.csv")
convention_name_2022_2023_df = pd.read_csv(convention_name_2022_2023, index_col ="Booked On", infer_datetime_format=True)
convention_name_2022_2023_df.head()

In [ ]:
combined_convention_name_2018_2023 = pd.concat([convention_name_2018_2019_df,convention_name_2020_2021_df, convention_name_2022_2023_df], axis=0)
combined_convention_name_2018_2023

In [ ]:
unique_convention_names = combined_convention_name_2018_2023["Notes"].unique()
unique_convention_names

In [ ]:
combined_convention_name_2018_2023["Notes"] = combined_convention_name_2018_2023["Notes"].str.lower().str.strip()

In [ ]:
most_common_conventions= combined_convention_name_2018_2023["Notes"].value_counts().head(40).index.tolist()
most_common_conventions


In [ ]:
notes_column = combined_convention_name_2018_2023['Notes']


In [ ]:
check_in_column = combined_convention_name_2018_2023["Check In"]

In [ ]:
unique_notes = notes_column.dropna().astype(str).unique()


In [ ]:
def clean_date_string(date_string):
    try:
        # Use dateutil.parser to parse the date
        date_obj = parser.parse(str(date_string))
        return date_obj.strftime("%m/%d/%Y")
    except (parser.ParserError, TypeError, ValueError):
        return None

In [ ]:
def extract_acronym(note):
    # Use regular expression to find uppercase letters in the note
    # and join them to form an acronym
    acronym_match = re.findall(r'\b[A-Z]+\b', note)
    return ''.join(acronym_match) if acronym_match else None

In [ ]:
def are_acronyms_similar(acronyms1, acronyms2):
    # Check if there is any common acronym between acronyms1 and acronyms2
    return any(acronym in acronyms2 for acronym in acronyms1) or any(acronym in acronyms1 for acronym in acronyms2)

In [ ]:
def are_notes_similar(note1, note2):
    # Extract acronyms from both notes
    acronym1 = extract_acronym(note1)
    acronym2 = extract_acronym(note2)
    
    return (acronym1 and acronym2 and acronym1 == acronym2) or note1 in note2 or note2 in note1

In [ ]:
def are_similar(note1, note2, check_in1, check_in2_series):
    try:
        check_in1_date = parser.parse(clean_date_string(check_in1))

        # Split the comma-separated date strings
        check_in2_dates = check_in2_series.str.split(',', expand=True).stack().reset_index(level=1, drop=True)

        # Convert the Series to a list and iterate over it
        for check_in2 in check_in2_dates.dropna().tolist():
            check_in2_date = parser.parse(clean_date_string(check_in2))
            check_in_similarity = abs((check_in1_date - check_in2_date).days) < 7

            # Combine both criteria to determine overall similarity
            if are_notes_similar(note1, note2) and check_in_similarity:
                return True
    except (parser.ParserError, TypeError, ValueError):
        # Handle parsing errors, if any
        pass

    return False


In [ ]:
similar_values_dict = {}

In [ ]:
for note in unique_notes:
    mask = notes_column.apply(lambda x: are_similar(note, x, combined_convention_name_2018_2023.loc[notes_column == note, 'Check In'].iloc[0], check_in_column))
    similar_values = combined_convention_name_2018_2023.loc[mask, 'Notes'].tolist()
    similar_values_dict[note] = similar_values

In [ ]:
for note, similar_values in similar_values_dict.items():
    print(f"Values for '{note}': {similar_values}")

# Sort the dictionary by keys
sorted_similar_values_dict = dict(sorted(similar_values_dict.items()))

# Print the sorted values
for note, similar_values in sorted_similar_values_dict.items():
    print(f"Similar values for '{note}': {similar_values}")


In [ ]:
similar_values_count = {}



for note, similar_values in similar_values_dict.items():
    count = len(similar_values)
    similar_values_count[note] = count
    print(f"Count of similar values for '{note}': {count}")

In [ ]:
sorted_counts = dict(sorted(similar_values_count.items(), key=lambda item: item[1], reverse=True))


print("\nSorted counts:")
for note, count in sorted_counts.items():
    print(f"Count of similar values for '{note}': {count}")

In [ ]:
similar_values_info = {}

for note, similar_values in similar_values_dict.items():
    count = len(similar_values)
    check_in_dates = combined_convention_name_2018_2023.loc[notes_column.isin(similar_values), 'Check In'].unique()
    similar_values_info[note] = {"count": count, "check_in_dates": check_in_dates}

# Create a DataFrame from the dictionary
similar_values_df = pd.DataFrame.from_dict(similar_values_info, orient='index')

# Sort the DataFrame by counts in descending order
similar_values_df = similar_values_df.sort_values(by='count', ascending=False)

# Print the sorted DataFrame
print("\nSorted DataFrame:")
print(similar_values_df.head(50))



In [ ]:
# Create a DataFrame from the dictionary
similar_values_df = pd.DataFrame.from_dict(similar_values_info, orient='index')

# Rename the columns for clarity
similar_values_df.columns = ['Count', 'Check In Dates']

# Sort the DataFrame by counts in descending order
similar_values_df = similar_values_df.sort_values(by='Count', ascending=False)

# Print the sorted DataFrame
print("\nSorted DataFrame:")
print(similar_values_df.head(50))


In [ ]:
# Create a DataFrame from the dictionary
similar_values_df = pd.DataFrame.from_dict(similar_values_info, orient='index')

# Rename the columns for clarity
similar_values_df.columns = ['Count', 'Check In Dates']

# Sort the DataFrame by counts in descending order
similar_values_df = similar_values_df.sort_values(by='Count', ascending=False)

# Print the sorted DataFrame
print("\nSorted DataFrame:")
print(similar_values_df.head(50))

# Save the DataFrame to a CSV file
# similar_values_df.to_csv('similar_values_output.csv')


In [ ]:
import pandas as pd

# Assuming you have already created the similar_values_df DataFrame

# Convert the "Check In Dates" column to datetime
similar_values_df['Check In Dates'] = pd.to_datetime(similar_values_df['Check In Dates'], errors='coerce')

# Sort the DataFrame by counts in descending order and then by "Check In Dates" in descending order
similar_values_df = similar_values_df.sort_values(by=['Count', 'Check In Dates'], ascending=[False, False])

# Print the sorted DataFrame
print("\nSorted DataFrame:")
print(similar_values_df.head(50))

# Save the DataFrame to a CSV file
# similar_values_df.to_csv('similar_values_output_sorted.csv')


In [ ]:
filtered_df = combined_convention_name_2018_2023[combined_convention_name_2018_2023["Notes"].isin(most_common_conventions)]
filtered_df

In [ ]:
sorted_by_most_common_names = filtered_df.sort_values(by="Notes", key=lambda x: x.map(most_common_conventions.index))
sorted_by_most_common_names.head(20)                                                                    

In [ ]:
sorted_by_most_common_names.drop_duplicates(subset="Notes", keep='first', inplace=True)
sorted_by_most_common_names.head(20)

In [ ]:
null_check_in = sorted_by_most_common_names['Check In'].isnull().value_counts()
null_check_in

In [ ]:
sorted_by_most_common_names.dropna(subset=['Check In'], inplace=True)

In [ ]:
sorted_by_most_common_names['Check In'] = pd.to_datetime(sorted_by_most_common_names['Check In'], errors="coerce")


In [ ]:
sorted_by_most_common_names.dtypes

In [ ]:
new_index = pd.to_datetime(sorted_by_most_common_names.index, errors='coerce')


In [ ]:
sorted_by_most_common_names = sorted_by_most_common_names.set_index(new_index)


In [ ]:
time_difference = sorted_by_most_common_names.index - sorted_by_most_common_names['Check In']


In [ ]:
time_difference_days = time_difference.astype('timedelta64[D]')


In [ ]:
sorted_by_most_common_names['Time Difference (Days)'] = time_difference_days.astype(int)


In [ ]:
sorted_by_most_common_names.drop(columns=['Time Difference'], inplace=True)


In [ ]:
sorted_by_most_common_names

In [ ]:
sorted_by_days_reserved_before_checkin_date_df = sorted_by_most_common_names.sort_values(by='Time Difference (Days)')


In [ ]:
sorted_by_days_reserved_before_checkin_date_df